In [2]:
# cd 'C:/Users/arthu/Documents/GitHub/TCC_Stock_Predictions/'

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
%load_ext lab_black

<IPython.core.display.Javascript object>

# Window Process 

In [3]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU

LeakyReLU = LeakyReLU(alpha=0.1)
import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

<IPython.core.display.Javascript object>

In [6]:
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

<IPython.core.display.Javascript object>

In [4]:
# 60%
PATH_TREINO = "../Data/3_Gold/Treino_all_stocks.csv"
DF_TREINO = pd.read_csv(PATH_TREINO, sep=",")
train_dataset = DF_TREINO
train_dataset = train_dataset.drop(
    [
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
train_labels = train_dataset.pop("ibova_5")
# train_dataset = train_dataset.iloc[:476]

# 20%
PATH_VALIDACAO = "../Data/3_Gold/Validacao_all_stocks.csv"
DF_VALIDACAO = pd.read_csv(PATH_VALIDACAO, sep=",")
valid_dataset = DF_VALIDACAO
valid_dataset = valid_dataset.drop(
    [
        "ibova_5",
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
# valid_labels = valid_dataset.pop('ibova_5')

# 20%
PATH_TESTE = "../Data/3_Gold/Teste_all_stocks.csv"
DF_TESTE = pd.read_csv(PATH_TESTE, sep=",")
test_dataset = DF_TESTE
test_dataset = test_dataset.drop(
    [
        "oil_5",
        "usd_5",
        "abev_5",
        "jbs_5",
        "petr_5",
        "vale_5",
    ],
    axis=1,
)
test_labels = test_dataset.pop("ibova_5")

<IPython.core.display.Javascript object>

In [8]:
# Load dataset
trainSet = pd.read_csv("../Project-hyper/train.csv")
# Feature generation: training data
train = trainSet.drop(columns=["Name", "Ticket", "Cabin"])
train = train.dropna(axis=0)
train = pd.get_dummies(train)
# train validation split
train_dataset, train_labels, y_train, y_val = train_test_split(
    train.drop(columns=["PassengerId", "Survived"], axis=0),
    train["Survived"],
    test_size=0.2,
    random_state=111,
    stratify=train["Survived"],
)

<IPython.core.display.Javascript object>

In [9]:
# Create function
def nn_cl_bo(neurons, activation, optimizer, learning_rate, batch_size, epochs):
    optimizerL = [
        "SGD",
        "Adam",
        "RMSprop",
        "Adadelta",
        "Adagrad",
        "Adamax",
        "Nadam",
        "Ftrl",
        "SGD",
    ]
    optimizerD = {
        "Adam": Adam(lr=learning_rate),
        "SGD": SGD(lr=learning_rate),
        "RMSprop": RMSprop(lr=learning_rate),
        "Adadelta": Adadelta(lr=learning_rate),
        "Adagrad": Adagrad(lr=learning_rate),
        "Adamax": Adamax(lr=learning_rate),
        "Nadam": Nadam(lr=learning_rate),
        "Ftrl": Ftrl(lr=learning_rate),
    }
    activationL = [
        "relu",
        "sigmoid",
        "softplus",
        "softsign",
        "tanh",
        "selu",
        "elu",
        "exponential",
        LeakyReLU,
        "relu",
    ]
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)

    def nn_cl_fun():
        opt = Adam(lr=learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation="sigmoid"))
        nn.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
        return nn

    es = EarlyStopping(monitor="accuracy", mode="max", verbose=0, patience=20)
    nn = KerasClassifier(
        build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0
    )
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(
        nn,
        train_dataset,
        y_train,
        scoring=score_acc,
        cv=kfold,
        fit_params={"callbacks": [es]},
    ).mean()
    return score

<IPython.core.display.Javascript object>

In [10]:
# Set paramaters
params_nn = {
    "neurons": (10, 100),
    "activation": (0, 9),
    "optimizer": (0, 7),
    "learning_rate": (0.01, 1),
    "batch_size": (200, 1000),
    "epochs": (20, 100),
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
482/482 [==============================] - 1s 2ms/step
|  1        |  0.5151   |  5.51     |  335.3    |  54.88    |  0.7716   |  36.58    |  1.044    |
482/482 [==============================] - 1s 3ms/step
|  2        |  0.5719   |  0.2023   |  536.2    |  39.09    |  0.3443   |  99.16    |  1.664    |
482/482 [==============================] - 1s 2ms/step
|  3        |  0.5719   |  0.7307   |  735.7    |  69.7     |  0.2815   |  51.96    |  0.8286   |
482/482 [==============================] - 2s 3ms/step
|  4        |  0.5431   |  0.6656   |  920.6    |  83.52    |  0.8422   |  83.37    |  6.937    |
482/482 [==============================] - 2s 3ms/step
|  5        |  0.7654   |  5.195    |  851.0    |  53.71    |  0.03717  |  50.87    |  0.7373   |
482/482 [==============================

<IPython.core.display.Javascript object>

In [11]:
params_nn_ = nn_bo.max["params"]
activationL = [
    "relu",
    "sigmoid",
    "softplus",
    "softsign",
    "tanh",
    "selu",
    "elu",
    "exponential",
    LeakyReLU,
    "relu",
]
params_nn_["activation"] = activationL[round(params_nn_["activation"])]
params_nn_

{'activation': 'selu',
 'batch_size': 851.0135336291902,
 'epochs': 53.7054301919375,
 'learning_rate': 0.037173480215022196,
 'neurons': 50.872297884262295,
 'optimizer': 0.7372825972056519}

<IPython.core.display.Javascript object>

## Tune Layers

In [19]:
# Create function
def nn_cl_bo2(
    neurons,
    activation,
    optimizer,
    learning_rate,
    batch_size,
    epochs,
    layers1,
    layers2,
    normalization,
    dropout,
    dropout_rate,
):
    optimizerL = [
        "SGD",
        "Adam",
        "RMSprop",
        "Adadelta",
        "Adagrad",
        "Adamax",
        "Nadam",
        "Ftrl",
        "SGD",
    ]
    optimizerD = {
        "Adam": Adam(lr=learning_rate),
        "SGD": SGD(lr=learning_rate),
        "RMSprop": RMSprop(lr=learning_rate),
        "Adadelta": Adadelta(lr=learning_rate),
        "Adagrad": Adagrad(lr=learning_rate),
        "Adamax": Adamax(lr=learning_rate),
        "Nadam": Nadam(lr=learning_rate),
        "Ftrl": Ftrl(lr=learning_rate),
    }
    activationL = [
        "relu",
        "sigmoid",
        "softplus",
        "softsign",
        "tanh",
        "selu",
        "elu",
        "exponential",
        LeakyReLU,
        "relu",
    ]
    neurons = round(neurons)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def nn_cl_fun():
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=10, activation=activation))
        if normalization > 0.5:
            nn.add(BatchNormalization())
        for i in range(layers1):
            nn.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            nn.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation="sigmoid"))
        nn.compile(
            loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"]
        )
        return nn

    es = EarlyStopping(monitor="accuracy", mode="max", verbose=0, patience=20)
    nn = KerasClassifier(
        build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0
    )
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(
        nn,
        train_dataset,
        y_train,
        scoring=score_acc,
        cv=kfold,
        fit_params={"callbacks": [es]},
    ).mean()
    return score

IndentationError: expected an indented block (3049273778.py, line 15)

ERROR:root:Cannot parse: 15:0: optimizerL = [
Traceback (most recent call last):
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\__init__.py", line 1163, in format_str
    dst_contents = _format_str_once(src_contents, mode=mode)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\__init__.py", line 1173, in _format_str_once
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "c:\Users\arthu\.conda\envs\pystock\lib\site-packages\black\parsing.py", line 128, in lib2to3_parse
    raise exc from None
black.parsing.InvalidInput: Cannot parse: 15:0: optimizerL = [
ERROR:root:Cannot parse: 15:0: optimizerL = [
Traceback (most recent cal

In [13]:
params_nn2 = {
    "neurons": (10, 100),
    "activation": (0, 9),
    "optimizer": (0, 7),
    "learning_rate": (0.01, 1),
    "batch_size": (200, 1000),
    "epochs": (20, 100),
    "layers1": (1, 3),
    "layers2": (1, 3),
    "normalization": (0, 1),
    "dropout": (0, 1),
    "dropout_rate": (0, 0.3),
}
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo2, params_nn2, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
482/482 [==============================] - 1s 3ms/step


KeyboardInterrupt: 

<IPython.core.display.Javascript object>

#### Executing

In [14]:
params_nn_ = nn_bo.max["params"]
learning_rate = params_nn_["learning_rate"]
activationL = [
    "relu",
    "sigmoid",
    "softplus",
    "softsign",
    "tanh",
    "selu",
    "elu",
    "exponential",
    LeakyReLU,
    "relu",
]
params_nn_["activation"] = activationL[round(params_nn_["activation"])]
params_nn_["batch_size"] = round(params_nn_["batch_size"])
params_nn_["epochs"] = round(params_nn_["epochs"])
params_nn_["layers1"] = round(params_nn_["layers1"])
params_nn_["layers2"] = round(params_nn_["layers2"])
params_nn_["neurons"] = round(params_nn_["neurons"])
optimizerL = [
    "Adam",
    "SGD",
    "RMSprop",
    "Adadelta",
    "Adagrad",
    "Adamax",
    "Nadam",
    "Ftrl",
    "Adam",
]
optimizerD = {
    "Adam": Adam(lr=learning_rate),
    "SGD": SGD(lr=learning_rate),
    "RMSprop": RMSprop(lr=learning_rate),
    "Adadelta": Adadelta(lr=learning_rate),
    "Adagrad": Adagrad(lr=learning_rate),
    "Adamax": Adamax(lr=learning_rate),
    "Nadam": Nadam(lr=learning_rate),
    "Ftrl": Ftrl(lr=learning_rate),
}
params_nn_["optimizer"] = optimizerD[optimizerL[round(params_nn_["optimizer"])]]
params_nn_

KeyError: 'params'

<IPython.core.display.Javascript object>

In [18]:
# Fitting Neural Network
def nn_cl_fun():

    nn = Sequential()
    nn.add(
        Dense(params_nn_["neurons"], input_dim=10, activation=params_nn_["activation"])
    )
    if params_nn_["normalization"] > 0.5:
        nn.add(BatchNormalization())
    for i in range(params_nn_["layers1"]):
        nn.add(Dense(params_nn_["neurons"], activation=params_nn_["activation"]))
    if params_nn_["dropout"] > 0.5:
        nn.add(Dropout(params_nn_["dropout_rate"], seed=123))
    for i in range(params_nn_["layers2"]):
        nn.add(Dense(params_nn_["neurons"], activation=params_nn_["activation"]))
        nn.add(Dense(1, activation="sigmoid"))
        nn.compile(
            loss="binary_crossentropy",
            optimizer=params_nn_["optimizer"],
            metrics=["accuracy"],
        )
    return nn


es = EarlyStopping(monitor="accuracy", mode="max", verbose=0, patience=20)
nn = KerasClassifier(
    build_fn=nn_cl_fun,
    epochs=params_nn_["epochs"],
    batch_size=params_nn_["batch_size"],
    verbose=0,
)
nn.fit(X_train, y_train, validation_data=(X_val, y_val), verbose=1)

NameError: name 'X_train' is not defined

<IPython.core.display.Javascript object>